## CCF: 企业非法集资风险预测

In [1040]:
import time
import math
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb
from util import *

t_base_info = "./dataset/train/base_info.csv"
t_annual_report_info = "./dataset/train/annual_report_info.csv"
t_tax_info = "./dataset/train/tax_info.csv"
t_change_info = "./dataset/train/change_info.csv"
t_news_info = "./dataset/train/news_info.csv"
t_other_info = "./dataset/train/other_info.csv"
t_entprise_info = "./dataset/train/entprise_info.csv" # train labeled data

In [1041]:
print("Reading...")
# base_info = pd.read_csv('dataset/train/base_info.csv')

Reading...


In [1042]:
# base_info.describe()

### entprise_info

In [1043]:
df_label_info = pd.read_csv(t_entprise_info, header=0)

df_label_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14865 entries, 0 to 14864
Data columns (total 2 columns):
id       14865 non-null object
label    14865 non-null int64
dtypes: int64(1), object(1)
memory usage: 232.4+ KB


In [1044]:
df_label_info['label'].unique()
df_label_info['id'].unique()
df_label_info['label'].value_counts()
df_label_info['id'].value_counts()


755db3b5c5f74eb4b78c720275404597b8c157fd00ae249b    1
216bd2aaf4d079241368de7714d0f91a68a79f3e0d8720f9    1
216bd2aaf4d0792471dff9ca9cc71c5913685db426558edc    1
beb4aaaa89e0a0aeaa3b85f652794fa14101a81d2a14b6ed    1
f000950527a6feb6cd58d7abbb90ea8e28bf8fd83ee5ea84    1
                                                   ..
755db3b5c5f74eb49bb393a31c5aed57ab98f3364446496b    1
f000950527a6feb68529abe82c9dbf806d452cbcba114369    1
9c7fa510616a68304d2aeac3071cecd8d26f5c64f8606c48    1
beb4aaaa89e0a0aea8b6620715a4038887ecfd00a6fb9fa1    1
8f19388cfd6e2b9cda01861a820fd1c87597e37f75260200    1
Name: id, Length: 14865, dtype: int64

### other_info.csv

In [1045]:
# TODO: 特征空值过多 如何处理？ #
df_other_info = pd.read_csv(t_other_info, header=0)
df_other_info['legal_judgment_num'].value_counts()
df_other_info.shape
df_other_info.head(5)

,id,legal_judgment_num,brand_num,patent_num
0,f000950527a6feb6d340f91da09e61347d8200cd2f0d1602,4.0,NaN,NaN
1,f000950527a6feb608dd9322b74a99f60851207f36a3c94c,1.0,NaN,NaN
2,d8071a739aa75a3b9f23966f8dae78fd226c272515b9c255,2.0,NaN,NaN
3,216bd2aaf4d079242209b1496f81a36c7abed9dd0bb65ed3,NaN,1.0,NaN
4,e9f7b28ec10e0470de9631c789f49acdd4e7cf9ed6db094b,NaN,2.0,NaN


In [1046]:
# 统计空值的个数
null_cnt_lj = df_other_info['legal_judgment_num'].isnull().sum(axis=0)
null_cnt_br = df_other_info['brand_num'].isnull().sum(axis=0)
null_cnt_pt = df_other_info['patent_num'].isnull().sum(axis=0)
print(null_cnt_lj, null_cnt_br, null_cnt_pt)


884 981 1494


In [1047]:
training_set = pd.merge(df_label_info, df_other_info, how='left', on='id')
# 释放内存
del df_label_info
del df_other_info
# training_set['id'].value_counts()
training_set = training_set.groupby(['id'], as_index=False).mean()
# training_set[training_set['id'] == "f000950527a6feb63702b1f6c1dabe5ea196d320bbbff425"]
training_set.shape

(14865, 5)

### news_info.csv

In [1048]:
#  TODO: 随着时间变化，news的态度不同 但是标签只能是一个#
df_news_info = pd.read_csv(t_news_info, header=0)

dict_atitude = {"积极": 0, "中立": 1, "消极": 2}
df_news_info['positive_negtive'] = df_news_info['positive_negtive'].map(lambda x : dict_atitude[x])
# public date 转变成迄今为止发生时间
cmp_date = "2020-10-09"
from util import *

# df_news_info['public_date'] = df_news_info['public_date'].map(lambda x: check_date(x))
# df_news_info['public_date'].value_counts()
# 处理"xx小时前" 数据统一为昨天更新 
def handle_public_date(str):
    if(check_date(str) == False):
        return 1
    else:
        return days(cmp_date, str)
df_news_info['public_date'] = df_news_info['public_date'].map(lambda x: handle_public_date(x))
df_news_info.shape

time data '4小时前' does not match format '%Y-%m-%d'
time data '14小时前' does not match format '%Y-%m-%d'
time data '25分钟前' does not match format '%Y-%m-%d'
time data '30分钟前' does not match format '%Y-%m-%d'
time data '1小时前' does not match format '%Y-%m-%d'
time data '2小时前' does not match format '%Y-%m-%d'
time data '2小时前' does not match format '%Y-%m-%d'
time data '8小时前' does not match format '%Y-%m-%d'
time data '9小时前' does not match format '%Y-%m-%d'
time data '22小时前' does not match format '%Y-%m-%d'
time data '23小时前' does not match format '%Y-%m-%d'
time data '6小时前' does not match format '%Y-%m-%d'
time data '5小时前' does not match format '%Y-%m-%d'
time data '8小时前' does not match format '%Y-%m-%d'
time data '8小时前' does not match format '%Y-%m-%d'
time data '9小时前' does not match format '%Y-%m-%d'


(10518, 3)

In [1049]:
training_set = pd.merge(training_set, df_news_info, how='left', on='id')
training_set['id'].value_counts()

f000950527a6feb63702b1f6c1dabe5ea196d320bbbff425    1193
d8071a739aa75a3b04c9a1789bd657ef83e10c02b1a9fef5     290
f000950527a6feb6194eb25023f715fa9ec0166d20cc8aed     215
f000950527a6feb6c0db441978864a6a0f04668a61d0bab6     202
f000950527a6feb6375809f26c0e7fa103f56259da2e53ac     160
                                                    ... 
f000950527a6feb68c162c78e6dd292e400fb157f49e6d17       1
ed38190adf12fcebf79b42d7f23db1b7de5c010460c12798       1
f000950527a6feb6927d6cf7c99ef548f9ca363226d3f718       1
516ab81418ed215dcbbfaf81eed615600d8fd47c9cc4fda0       1
8f19388cfd6e2b9cda01861a820fd1c87597e37f75260200       1
Name: id, Length: 14865, dtype: int64

In [1050]:
# 一个id对应不同时间的不同新闻评价
tmp = training_set[training_set['id'] == "f000950527a6feb63702b1f6c1dabe5ea196d320bbbff425"]
tmp.head(100)
training_set.shape
training_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19407 entries, 0 to 19406
Data columns (total 7 columns):
id                    19407 non-null object
label                 19407 non-null int64
legal_judgment_num    3258 non-null float64
brand_num             2924 non-null float64
patent_num            2334 non-null float64
positive_negtive      4930 non-null float64
public_date           4930 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 1.2+ MB


### change_info.csv

In [1051]:
# TODO: 变更信息如何处理  这里直接删除 只保留变更时间 #
df_change_info = pd.read_csv(t_change_info, header=0)
# df_change_info['bqyf'] = df_change_info[['bgq', 'bgh']].apply(lambda x: x['bgq'] == x['bgh'], axis=1)
# del df_change_info[]
# df_change_info.shape
del df_change_info['bgxmdm']
del df_change_info['bgq']
del df_change_info['bgh']
df_change_info['bgrq'] = df_change_info['bgrq'].apply(np.log)
# 平滑 bgrq:变更日期
df_change_info.head()

,id,bgrq
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,30.636238
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,30.636238
2,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,30.635224
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,30.635224
4,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,30.635224


### tax_info.csv


In [1052]:
# TODO: 很多统计信息， 需要去提取特征 #
df_tax_info = pd.read_csv(t_tax_info, header=0)
df_tax_info['TAX_CATEGORIES'].value_counts()
df_tax_info['TAX_ITEMS'].value_counts()
df_tax_info['TAX_AMOUNT'].value_counts()
df_tax_info['TAXATION_BASIS'].value_counts()

df_tax_info['TAX_DAYS'] = df_tax_info[['START_DATE', 'END_DATE']].apply(lambda x : days_v1(x['END_DATE'], x['START_DATE']), axis=1)

del df_tax_info['START_DATE']
del df_tax_info['END_DATE']

tax_cg_dict = {}
cnt = 0
for e in df_tax_info['TAX_CATEGORIES']:
    if e in tax_cg_dict.unique(): continue
    else:
        tax_cg_dict[e] = cnt 
        cnt += 1

df_tax_info['TAX_CATEGORIES'] = df_tax_info['TAX_CATEGORIES'].map(tax_cg_dict)

tax_it_dict = {}
cnt = 0
for e in df_tax_info['TAX_ITEMS']:
    if e in tax_it_dict.unique(): continue
    else:
        tax_it_dict[e] = cnt 
        cnt += 1
df_tax_info['TAX_ITEMS'] = df_tax_info['TAX_ITEMS'].map(tax_it_dict)
df_tax_info['TAXATION_BASIS'] = df_tax_info['TAXATION_BASIS'].apply(np.log)

df_tax_info.head()


AttributeError: 'dict' object has no attribute 'unique'

In [900]:
training_set = pd.merge(training_set, df_tax_info, how='left', on='id')
# training_set['id'].value_counts()
# training_set.info()
training_set.shape

(205337, 14)

### annual_report_info.csv

In [901]:
# 如何从大量的空值中 提取到有用信息 #
df_anreport_info = pd.read_csv(t_annual_report_info, header=0)
# df_anreport_info['ANCHEYEAR'].value_counts()
# df_anreport_info['STATE'].value_counts()
# df_anreport_info.iloc[:, 0:13] 
# df_anreport_info['FUNDAM'].value_counts()

dict_year = {"2015.0": 0, "2016.0": 1, "2017.0": 2, "2018.0": 3}
df_anreport_info['ANCHEYEAR'] = df_anreport_info['ANCHEYEAR'].map(lambda x : dict_year[str(x)])

# df_anreport_info['MEMNUM'].value_counts() 
# df_anreport_info['MEMNUM'].isnull().sum(axis=0) #空值太多
del df_anreport_info['MEMNUM']
df_anreport_info.head()
# df_anreport_info['FARNUM'].value_counts() 
# df_anreport_info['FARNUM'].isnull().sum(axis=0) #空值太多
del df_anreport_info['FARNUM']

# df_anreport_info['ANNNEWMEMNUM'].value_counts() 
# df_anreport_info['ANNNEWMEMNUM'].isnull().sum(axis=0) #空值太多
del df_anreport_info['ANNNEWMEMNUM']

# df_anreport_info['ANNREDMEMNUM'].value_counts() 
# df_anreport_info['ANNREDMEMNUM'].isnull().sum(axis=0) #空值太多
del df_anreport_info['ANNREDMEMNUM']

# df_anreport_info['EMPNUM'].value_counts() 
# df_anreport_info['EMPNUMSIGN'].value_counts() 

# df_anreport_info['BUSSTNAME'].fillna("无")
df_anreport_info['BUSSTNAME'].value_counts() 
dict_bsnm = { "开业": 1, "歇业": 2, "停业": 3, "清算": 4}
df_anreport_info['BUSSTNAME'] = df_anreport_info['BUSSTNAME'].map(dict_bsnm)
df_anreport_info['BUSSTNAME'].fillna(0, inplace=True)

df_anreport_info['COLGRANUM'].value_counts() 
df_anreport_info['RETSOLNUM'].value_counts() 
df_anreport_info['DISPERNUM'].value_counts() 
df_anreport_info['UNENUM'].value_counts() 
df_anreport_info['COLEMPLNUM'].value_counts() 
df_anreport_info['RETEMPLNUM'].value_counts() 
df_anreport_info['DISEMPLNUM'].value_counts() 
df_anreport_info['WEBSITSIGN'].value_counts() 
df_anreport_info['FORINVESTSIGN'].value_counts() 
df_anreport_info['STOCKTRANSIGN'].value_counts() 
df_anreport_info['PUBSTATE'].value_counts() 
df_anreport_info.head()


,id,ANCHEYEAR,STATE,FUNDAM,EMPNUM,EMPNUMSIGN,BUSSTNAME,COLGRANUM,RETSOLNUM,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2,2.0,5.0,10.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,3,2.0,2.0,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
2,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2,2.0,NaN,4.0,2.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
3,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,3,2.0,NaN,3.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
4,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,2,2.0,5.0,10.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0


In [1054]:
training_set = pd.merge(training_set, df_anreport_info, how='left', on='id')
training_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31304 entries, 0 to 31303
Data columns (total 25 columns):
id                    31304 non-null object
label                 31304 non-null int64
legal_judgment_num    10178 non-null float64
brand_num             8277 non-null float64
patent_num            7617 non-null float64
positive_negtive      13845 non-null float64
public_date           13845 non-null float64
ANCHEYEAR             18128 non-null float64
STATE                 18126 non-null float64
FUNDAM                1173 non-null float64
EMPNUM                18124 non-null float64
EMPNUMSIGN            16951 non-null float64
BUSSTNAME             18128 non-null float64
COLGRANUM             7769 non-null float64
RETSOLNUM             7769 non-null float64
DISPERNUM             7769 non-null float64
UNENUM                7769 non-null float64
COLEMPLNUM            7769 non-null float64
RETEMPLNUM            7769 non-null float64
DISEMPLNUM            7769 non-null float64
UNEE

### base_info.csv

In [1039]:
df_base_info = pd.read_csv(t_base_info, header=0)

df_base_info['oplocdistrict'].value_counts() 
dict_oplocdistrict = {}
cnt = 0
for e in df_base_info['oplocdistrict'].unique():
    if e in dict_oplocdistrict: continue
    else:
        dict_oplocdistrict[e] = cnt 
        cnt += 1
# print(dict_oplocdistrict)
df_base_info['oplocdistrict'] = df_base_info['oplocdistrict'].map(dict_oplocdistrict)

df_base_info['industryphy'].value_counts() 
dict_industryphy = {}
cnt = 0
for e in df_base_info['industryphy'].unique():
    if e in dict_industryphy: continue
    else:
        dict_industryphy[e] = cnt 
        cnt += 1
# print(dict_industryphy)
df_base_info['industryphy'] = df_base_info['industryphy'].map(dict_industryphy)

df_base_info['industryco'].value_counts() 
dict_industryco = {}
cnt = 0
for e in df_base_info['industryco'].unique():
    if e in dict_industryco: continue
    else:
        dict_industryco[e] = cnt 
        cnt += 1
# print(dict_industryco)
df_base_info['industryco'] = df_base_info['industryco'].map(dict_industryco)

df_base_info['dom'].value_counts() 
del df_base_info['dom']
del df_base_info['opscope']

df_base_info['enttype'].value_counts() 
dict_enttype = {}
cnt = 0
for e in df_base_info['enttype'].unique():
    if e in dict_enttype: continue
    else:
        dict_enttype[e] = cnt 
        cnt += 1
# print(dict_enttype)
df_base_info['enttype'] = df_base_info['enttype'].map(dict_enttype)
# df_base_info['enttype'].fillna(0, inplace=True)


df_base_info['enttypeitem'].value_counts() 
dict_enttypeitem = {}
cnt = 0
for e in df_base_info['enttypeitem'].unique():
    if e in dict_enttypeitem: continue
    else:
        dict_enttypeitem[e] = cnt 
        cnt += 1
# print(dict_enttypeitem)
df_base_info['enttypeitem'] = df_base_info['enttypeitem'].map(dict_enttypeitem)

del df_base_info['opfrom']
del df_base_info['opto']


df_base_info['state'].value_counts()
dict_state = {}
cnt = 0
for e in df_base_info['state'].unique():
    if e in dict_state: continue
    else:
        dict_state[e] = cnt 
        cnt += 1
# print(dict_state)
df_base_info['state'] = df_base_info['state'].map(dict_state)

del df_base_info['orgid']
del df_base_info['jobid']

df_base_info['adbusign'].value_counts()
df_base_info['townsign'].value_counts()
df_base_info['regtype'].value_counts()
df_base_info['empnum'].value_counts()
df_base_info['compform'].value_counts()
df_base_info['parnum'].value_counts()
df_base_info['exenum'].value_counts()


df_base_info['opform'].value_counts()
dict_opform = {}
cnt = 0
for e in df_base_info['opform'].unique():
    if e in dict_opform: continue
    else:
        dict_opform[e] = cnt 
        cnt += 1
# print(dict_opform)
df_base_info['opform'] = df_base_info['opform'].map(dict_opform)

df_base_info['ptbusscope'].value_counts()
del df_base_info['ptbusscope']

df_base_info['venind'].value_counts()

df_base_info['enttypeminu'].value_counts()
dict_enttypeminu = {}
cnt = 0
for e in df_base_info['enttypeminu'].unique():
    if e in dict_enttypeminu: continue
    else:
        dict_enttypeminu[e] = cnt 
        cnt += 1
# print(dict_enttypeminu)
df_base_info['enttypeminu'] = df_base_info['enttypeminu'].map(dict_enttypeminu)
df_base_info['midpreindcode'].value_counts()
del df_base_info['midpreindcode']

df_base_info['protype'].value_counts() #空值太多
del df_base_info['protype']
df_base_info['oploc'].value_counts() 
del df_base_info['oploc']

df_base_info['regcap'].value_counts() 
df_base_info['reccap'].value_counts() 
df_base_info['forregcap'].value_counts() 
df_base_info['forreccap'].value_counts() 
df_base_info['congro'].value_counts() 
df_base_info['enttypegb'].value_counts() 
del df_base_info['enttypegb']

df_base_info.shape




(24865, 22)

In [1056]:
# training_set = pd.merge(training_set, df_base_info, how='left', on='id')
print(training_set.shape)
print(training_set.info())
print(training_set.head())


(31304, 46)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31304 entries, 0 to 31303
Data columns (total 46 columns):
id                    31304 non-null object
label                 31304 non-null int64
legal_judgment_num    10178 non-null float64
brand_num             8277 non-null float64
patent_num            7617 non-null float64
positive_negtive      13845 non-null float64
public_date           13845 non-null float64
ANCHEYEAR             18128 non-null float64
STATE                 18126 non-null float64
FUNDAM                1173 non-null float64
EMPNUM                18124 non-null float64
EMPNUMSIGN            16951 non-null float64
BUSSTNAME             18128 non-null float64
COLGRANUM             7769 non-null float64
RETSOLNUM             7769 non-null float64
DISPERNUM             7769 non-null float64
UNENUM                7769 non-null float64
COLEMPLNUM            7769 non-null float64
RETEMPLNUM            7769 non-null float64
DISEMPLNUM            7769 non-null 